In [1]:
# This is Main function.
# Extracting streaming data from Twitter, pre-processing, and loading into MySQL
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob
# Streaming With Tweepy 
# http://docs.tweepy.org/en/v3.4.0/streaming_how_to.html#streaming-with-tweepy


# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    '''
    Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
    https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
    '''
    
    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False


In [2]:
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [3]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="mysqlroot",
    database="TwitterDB",
    charset = 'utf8'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()
    



In [9]:
auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [10]:
print(credentials.API_KEY)

CAFqK58iwxU7PIKL4VWtGva8X


In [13]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()

According to Facebook, the amount of a man you are depends on how well you can back up a trailer. Never knew this until now.
Long: None, Lati: None
RT @dbongino: Companies succeed in pressuring Facebook to yield, join Twitter in censoring, Parler's rise explodes https://t.co/NVKNC7ZcmN
Long: None, Lati: None
RT @ClaraJeffery: Everybody who works at @Facebook should quit. Pull Zuckerberg’s name off our hospital and return his blood money.  @ChanZ…
Long: None, Lati: None
RT @jkac: ONTARIO MUSHROOM HUNTERS AND FORAGERS facebook group... thank you... https://t.co/mXYN2tWvGw
Long: None, Lati: None
RT @immarkwilliam: ---------------------------------------
The Mindset You Need When Running Facebook Ads
---------------------------------…
Long: None, Lati: None
Starbucks to pause social media ads after campaign calls for Facebook ad boycott https://t.co/5vgMvhnshB
Long: None, Lati: None
This company is utterly racist. They told me to beat it after spending my hard earned BLACK dollars and tryi

RT @ibrsalazar: The first time I have read an explanation of the social contract in Filipino. And it is written well, so I have shared it o…
Long: None, Lati: None
Join my partner and me at SKEEA (Strategic Keys to Educational &amp; Economic Advancement) this afternoon at 4 PM as we… https://t.co/ij88GxHTfn
Long: None, Lati: None
Still not sure if I'm going to try and sell these books or just bring them to a charity shop cuz I don't want to bo… https://t.co/d3GldOfaOe
Long: None, Lati: None
@JamesOKeefeIII @Facebook Mark Suckerberg should be considered an enemy of the American people. #DeleteFacebook
Long: None, Lati: None
RT @JencaBenca: Sunday 23rd June - today
7pm Biafra time
Listen to the naked #truth about #TheZooCalledNigeria by @MaziNnamdiKanu https://t…
Long: None, Lati: None
@Ribbyse223 @NormOrnstein I'm still waiting for Jared to get back to us on crowdsourcing his sister-in-law's father's Facebook friends.
Long: None, Lati: None
RT @davidlitt: True story: I had a bunch of Fa

RT @ZevShalev: Today is a good day to repost this Facebook Narativ from 2017 @NarativLive.  https://t.co/jfYTR32dZs
Long: None, Lati: None
RT @ninnyd101: Need a full list of who supported the anti-Semitic BLM 
Premier League
Twitter 
Ben &amp; Jerry’s
Apple
L’Oreal
Facebook
Yorkshi…
Long: None, Lati: None
RT @HoarseWisperer: New from Really American.

If you are on Facebook with a friend list that includes potential reachables, this seems sha…
Long: None, Lati: None
RT @wlctv_ca: Assault, battery &amp; attempted robberty commited by antifa/blm on @OANN's reporter @JackPosobiec in DC
earlier this evening.…
Long: None, Lati: None
RT @moshekasher: facebook has a cool algorithm allowing someone you haven’t thought about in a decade  to pop up in your feed and you get t…
Long: None, Lati: None
RT @MP_Muye: Dear @Twitter, @Facebook and all other platforms, the people you pay to do translation of Hausa to English and probably other…
Long: None, Lati: None
RT @takeactionch: (Safety)

Kevin Car

RT @MollyJongFast: Facebook acts in bad faith. Advertisers have figured that out.
Long: None, Lati: None
RT @meghanbuell: @ford_prefect42 Please share. @webetrees99 is a participant in #giveoutday2020. https://t.co/mvk9qcnNbq Thanks. #Transvisi…
Long: None, Lati: None
RT @roughinform: It's been a tough few months, but for 2 hours every Sunday, laying on my living room floor with a lager, watching the clou…
Long: None, Lati: None
Google Adwords and Facebook Ads will drive more visitors to your site in just a day or two. https://t.co/KW79pSMvon
Long: None, Lati: None
RT @WgarNews: @TrishBarron @georgina_116 @LennaLeprena @lynlinking @JonesHowdareyou @anne_clothier @Michz45 @WhteRbbnAdvocat @LadyPoop2 @ja…
Long: None, Lati: None
RT @HannahSnodgra10: PLEASE SHARE THIS VIDEO: for more information and ways to help please look at the links below.

July 5, 2020 (A March…
Long: None, Lati: None
RT @greg_doucette: 6️⃣6️⃣3️⃣ San Jose, CA: at least 4 active duty SJPD cops have been given paid vaca

Power of social media
Long: None, Lati: None
RT @JuddLegum: BREAKING: @Starbucks pulling advertising from Facebook (and all other social media platforms)

Starbucks is a major Facebook…
Long: None, Lati: None
RT @PastorIzeIyamu: THANK YOU FOR ELECTING ME. 

First and foremost, I give thanks to God Almighty for the successful conduct of the primar…
Long: None, Lati: None
Missouri Democratic mayor accused of intimidating defund police protesters after reading names, partial addresses o… https://t.co/sVe24GUkx8
Long: None, Lati: None
RT @dbongino: Companies succeed in pressuring Facebook to yield, join Twitter in censoring, Parler's rise explodes https://t.co/NVKNC7ZcmN
Long: None, Lati: None
.@JuddLegum is doing the Lord's work

Thank you Judd

Subscribe to his newsletter to support him!!
Long: None, Lati: None
RT @alfredwebre: Report: Facebook is taking down this Poster - Please share freely! 🙃😇🥰 https://t.co/W1WULSeo8O
Long: None, Lati: None
RT @karaswisher: I’d say great, but honestly

...

Watch and Admire! 👇 https://t.co/6COftuGJ1i
Long: None, Lati: None
RT @TheRickWilson: Help us reach 10 million views of this video in 24 hours: please share it with your friends across Twitter, Facebook and…
Long: None, Lati: None
RT @TimHannan: So fucking disgusting.  @benshapiro has a network of  Facebook pages exploiting racial bias, religious bigotry, and violence…
Long: None, Lati: None
RT @TheRickWilson: Help us reach 10 million views of this video in 24 hours: please share it with your friends across Twitter, Facebook and…
Long: None, Lati: None
Four San Jose police officers placed on leave during investigation into racist Facebook posts https://t.co/2eWcgezyKz
Long: None, Lati: None
RT @TheRickWilson: Help us reach 10 million views of this video in 24 hours: please share it with your friends across Twitter, Facebook and…
Long: None, Lati: None
RT @OsmoWal: @MayaAzar10 @OnlyHotAss @AdultBrazil @stu007gots @MySexyGirlsPics @real_jimmy @Nym_HD @HotDevilPussy @BoatsNToesTwo @SexyND

RT @dbongino: Companies succeed in pressuring Facebook to yield, join Twitter in censoring, Parler's rise explodes https://t.co/NVKNC7ZcmN
Long: None, Lati: None
#60MWG “60 Minutes with God” prayers holds tomorrow &amp; every Monday from 5am! 🙏🏽 Invite others.

Joining details:-
P… https://t.co/PYsFYFi1UD
Long: None, Lati: None
RT @MollyJongFast: Facebook acts in bad faith. Advertisers have figured that out.
Long: None, Lati: None
RT @slpng_giants: That Facebook is sending their executives to news shows to spin their inaction on hate and disinformation rather than con…
Long: None, Lati: None
RT @OhNoSheTwitnt: I posted something on Facebook on being afraid to walk down the street alone at night and 3 different dudes suggested le…
Long: None, Lati: None
RT @TheRickWilson: Help us reach 10 million views of this video in 24 hours: please share it with your friends across Twitter, Facebook and…
Long: None, Lati: None
Now we're heading in the right direction. Soon FaceBook will only be able

A huge thank you all the drivers for their continued support following todays brilliant double session.

Drivers if… https://t.co/lSrjPc6SAM
Long: None, Lati: None
RT @ritholtz: Due to rampant misinformation, violent threats, voter suppression + divisive hate speech, these companies have suspended adve…
Long: None, Lati: None
RT @TheRickWilson: Help us reach 10 million views of this video in 24 hours: please share it with your friends across Twitter, Facebook and…
Long: None, Lati: None
RT @TheRickWilson: Help us reach 10 million views of this video in 24 hours: please share it with your friends across Twitter, Facebook and…
Long: None, Lati: None
RT @Eugene_V_Dabbs: People are showing up from all over town
Live stream:

https://t.co/lTpQ0eI84z
Long: None, Lati: None
RT @FMHDSD: UPDATE: @npower_ng Online Application

Inline with the ongoing N-POWER Batch C online application which commenced Friday  June…
Long: None, Lati: None
RT @TheRickWilson: Help us reach 10 million views of this v

RT @PeterAskin1: Two adorable calico/tortie babies ID 623236 - 237 at #CobbCounty in #MariettaGA need pledges for rescue ($200 goal)! Save…
Long: None, Lati: None
RT @TheRickWilson: Help us reach 10 million views of this video in 24 hours: please share it with your friends across Twitter, Facebook and…
Long: None, Lati: None
RT @EconomicTimes: Mark Zuckerberg just became $7.2 billion poorer after a flurry of companies pulled advertising from Facebook Inc.’s netw…
Long: None, Lati: None
RT @TheRickWilson: Help us reach 10 million views of this video in 24 hours: please share it with your friends across Twitter, Facebook and…
Long: None, Lati: None
RT @hiitaylorblake: Karen goes Live on Facebook to discuss racism, inclusivity and the importance of unlearning old habits. https://t.co/if…
Long: None, Lati: None
RT @TurnUpGvngstar: Found this on facebook 😔
Our women are hurting bafwethu  😓💔

Watch until the end 😭🤮💔 https://t.co/Qln0dA6u9F
Long: None, Lati: None
RT @TheRickWilson: Help us re

ReadTimeoutError: HTTPSConnectionPool(host='stream.twitter.com', port=443): Read timed out.